In [1]:
import os 
import warnings 
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd 
import cv2 
from tqdm import tqdm 
import matplotlib.pyplot as plt 

from PIL import Image

import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from torch.autograd import Variable
from torch.utils.data import Dataset 
from torch.utils.data import DataLoader
import torchvision 
from torchvision import transforms
from torchvision.datasets import STL10
import timm

#데이터 로더 
from utils import prepare_dataloader

#어그먼테이션 
from utils import augmenter

#model 
from utils import ResnetEncoder,ProjectionHead,PredictionHead,NNCLR

#loss, nn 
from utils import nearest_neighbour,contrastive_loss 
from utils import NTXentLoss

from pytorch_lars import LARS 
from info_nce import InfoNCE

#memorybank 
from utils import NNMemoryBankModule

# Main 

In [2]:
# 하이퍼 파라미터 

shuffle_buffer = 5000 
labelelled_train_images = 5000 
unlabelled_images = 100000

temperature = 0.1 
queue_size = 10000
contrastive_augmenter = {
    "brightness" : 0.5, 
    "name" : "contrastive_augmenter",
    "scale" : (0.2,1.0)
}
classification_augmenter = {
    "brightness": 0.2,
    "name": "classification_augmenter",
    "scale": (0.5, 1.0),
}
input_shape = (96,96,3)
width = 128 
num_epochs = 500 
steps_per_epoch = 200 
batch_size= 48
learning_rate = 1e-3 
device = 'cuda:0'
feature_dimensions = 256 


def model_save(model,model_name):
    torch.save(model,f'save_models/{model_name}')

In [ ]:
#데이터 로더 로드 
label_train, unlabel_train, test = prepare_dataloader(batch_size=batch_size)

#Augmentation 로드 
contrastive_augmentation = augmenter(contrastive_augmenter)
classification_augmentation = augmenter(classification_augmenter)

#모델 로드 
model = NNCLR(device).to(device)
memory_bank = NNMemoryBankModule().to(device)

#compile 
#criterion = NTXentLoss()
#optimizer = torch.optim.Adam(lr=learning_rate,params=model.parameters())
criterion = InfoNCE()
optimizer = LARS(model.parameters(),lr=learning_rate,momentum=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=100,eta_min=0)

#Train 
for epoch in tqdm(range(num_epochs)):
    model.train() 
    epoch_loss = 0.
    n = 0 
    #이미지 데이터 로드 
    for (label_img,labels),(unlabel_img,_) in zip(label_train,unlabel_train):
        label_img,unlabel_img = label_img.to(device),unlabel_img.to(device)

        images = torch.concat((label_img,unlabel_img),axis=0).to(device)
        augmented_images_1 = contrastive_augmentation(images).to(device)
        augmented_images_2 = contrastive_augmentation(images).to(device)
    

        optimizer.zero_grad()

        z1,p1 = model(augmented_images_1)
        #z2,p2 = model(augmented_images_2)

        z1 = memory_bank(z0,update=False)
        z2 = memory_bank(z1,update=True)
        loss = 0.5 * (criterion(z0,p1) + criterion(z1,p0))
        

        loss.backward()
        optimizer.step() 
        epoch_loss += loss 
        n+=1 
    print(f'Epoch : {epoch} | loss : {epoch_loss/n}')
    if epoch == 0:
        best = epoch_loss/n
        model_save(model,model_name='best.pt')
    if epoch_loss/n < best:
        model_save(model,model_name='best.pt')
        print(f'model save at {epoch}')
    


# Classifier 
- Linear evaluation protocol on ImageNet
- A simple framework for contrastive learning
of visual representations


In [17]:
device = 'cuda:0'
model_path = './save_models/best.pt'
model = torch.load(model_path).to(device)
root = './Data'

from utils import Custom_Dset
def train_loader():
    label_train_stl10 = STL10(root=root,split='train')
    label_train_Dset = Custom_Dset(label_train_stl10)
    label_train_loader = DataLoader(label_train_Dset,batch_size=32,shuffle=True)
    return label_train_loader 
    
class Classifier(nn.Module):
    def __init__(self,encoder,num_features):
        super(Classifier,self).__init__()
        self.encoder = self.__init_encoder__(encoder) 
        self.fc1 = nn.Linear(in_features=2048,out_features=num_features)

    def __init_encoder__(self,encoder):
        for name, params in encoder.named_parameters():
            if name not in ['fc.weight', 'fc.bias']:
                params.requires_grad = False
        return encoder 

    def forward(self,x):
        x = self.encoder(x)
        x = self.fc1(x)
        return x     

def score(y_,laels):
    y_pred = torch.argmax(y_,dim=1).detach().cpu().numpy()
    y_true = labels.detach().cpu().numpy()
    print(f1_score(y_true,y_pred,average='macro'))        

In [27]:
model =  Classifier(model.encoder,10).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
train_load = train_loader()
num_epochs = 90


for epoch in range(num_epochs):
    model.train()
    train_loss = 0. 
    for imgs,labels in train_load:
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        y_ = model(imgs)

        loss = criterion(y_,labels)
        loss.backward()
        optimizer.step() 

        train_loss += loss 
    print(epoch)
    print(f'Train_loss : {train_loss/len(train_load)}')
    score(y_,labels)
    




0
Train_loss : 0.6488103270530701
0.6666666666666666
1
Train_loss : 0.46838608384132385
0.7999999999999999
2
Train_loss : 0.426209956407547
0.6296296296296295
3
Train_loss : 0.42263147234916687
0.688888888888889
4
Train_loss : 0.40125465393066406
0.5952380952380952
5
Train_loss : 0.3765296936035156
0.5625
6
Train_loss : 0.38359272480010986
0.5238095238095238
7
Train_loss : 0.3476734757423401
0.611111111111111
8
Train_loss : 0.35047218203544617
0.625
9
Train_loss : 0.34709423780441284
0.5
10
Train_loss : 0.3452361822128296
1.0
11
Train_loss : 0.3404563367366791
0.4285714285714285
12
Train_loss : 0.3218770921230316
0.6296296296296295
13
Train_loss : 0.32035157084465027
0.8333333333333333
14
Train_loss : 0.31566953659057617
0.6857142857142857
15
Train_loss : 0.3106578290462494
0.5238095238095238
16
Train_loss : 0.2897692322731018
1.0
17
Train_loss : 0.3033568561077118
0.5476190476190476
18
Train_loss : 0.2878589332103729
0.35185185185185186
19
Train_loss : 0.3044440448284149
0.61904761904